# Laboratorio 2: Programación Lineal Mixta (MA4702, otoño 2023)

Miércoles 19 de Abril.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 3 etapas. Un trabajo introductorio, una etapa presencial y otra no-presencial.
La nota global del laboratorio será 30 % (TI) + 40 % (TP) + 30 % (TNP), donde (TI) es la nota del trabajo introductorio, (TP) la nota del trabajo presencial y (TNP) la nota del trabajo no presencial.

El trabajo introductorio es de carácter estrictamente individual. La etapa presencial y no-presencial son de carácter grupal. 

# Laboratorio 2: Trabajo Presencial (TP)


## Instrucciones.

En la parte presencial del laboratorio, realice las siguientes actividades:

* Complete la celda “A. Datos grupo”.
* Descargue los archivos y funciones auxiliares de UCursos (todos deben quedar en el mismo directorio en que está almacenado este archivo).
* Complete la sección "B. preparación".
* Lea el enunciado y complete los ejercicios 1, 2 y 3.
* Envíe el archivo .ipynb por el módulo de tareas de UCursos en la TAREA: Laboratorio 2 presencial. 
* Plazo de entrega: **miércoles 19 de abril a las 18:00** (se descontará 0.5 pts. por cada 15 min o fracción de atraso).


<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cual cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## A. Datos del grupo

<div class="alert alert-block alert-success">

- Número o identificador del Grupo: 
    
- Integrante 1: XXX

- Integrante 2: YYY     </div>


## B. Preparación

Desempaque el `.zip` del laboratorio en su directorio de trabajo manteniendo la estructura de sub-directorios en que vienen empaquetados los archivos. Entre los archivos descargados encontrará `lab2TP-Structs.jl` que es esencialmente el mismo archivo de *struct*'s del trabajo introductorio. 

Usaremos el paquete "DelimitedFiles". A continuación, descomente las líneas pertinentes si no tiene instalado el respectivo paquete.

In [ ]:
import Pkg
#Pkg.add("DelimitedFiles")   # Para instalar DelimitedFiles
using DelimitedFiles
include("lab2TP-Structs.jl");

Los sub-directorios `dirTP1/`, `dirTP2/` y `dirTP3/` contienen archivos de instancias, un directorio por cada uno de los ejercicios a realizar en la etapa presencial. Cada instancia corresponde a un problema de empaquetamiento, es decir, de la forma:

$$
\begin{array}{rlllr}
(PCK) & \max_{x} c^Tx \\
s.a. & Ax\leq b, \\
& x_i\in\{0,1\}^n, \forall i\in [n]. \\
\end{array}
$$

donde $A\in\mathbb{N}^{m\times n}$, $b\in\mathbb{N}^m$ y $c\in\mathbb{N}^n$. 
La codificación de $(PCK)$ en un archivo de instancia consta
de $(m+1)$ líneas cada una consistente de $(n+1)$ enteros separados
por comas. 
La línea $i$, con $i\in [m]$,
representa la $i$-ésima fila de $A$ (i.e., $A_{i,\star}$) seguido de $b_i$.
La última línea representa las coordenadas del vector
$c$ seguidas de $-1$.



## Ejercicio 1: Branching

El archivo `lab2TP-BnB-P1.jl` contiene esencialmente el mismo código que el archivo `lab2TI-BnB.jl` del trabajo introductorio. Ambos programas aplican la técnica de BnB para resolver programas lineales enteros escogiendo la variable $x_i$ en la cual bifurcar para cada nodo del árbol como aquella de menor índice que tenga valor fraccional. La función responsable de calcular el referido índice es `find_branching_variable()`. Su código se incluye a continuación: 

```julia 
function find_branching_variable(sol::Array{Float64,1}, current_node::BBNode)
  # Only called on non-integer solution sol. 

  n_vars = length(sol)
  for s in 1:n_vars
    v = sol[s]
    if v > integer_precision && v < 1 - integer_precision && ! is_variable_branched_on(current_node, s)
      return s
    end
  end
  println(" -> Where shall I branch?")    # Warning message.
end
```
Otras estrategias de selección de variables para bifurcar son al azar (entre todas las variables fraccionales), de acuerdo a prioridades proveídas por el usuario, bifurcar en la variable más fraccional, y estrategias avanzadas como pseudo-costos, *lookahead*, *strong branching*, etc.

Ejecute a continuación la siguiente celda y observe el gran número de nodos del árbol BnB que se genera en cada caso (Nota: Para no exceder el límite de bytes posibles de desplegar como resultado de una ejecución de una celda de Jupyter Notebook, solo se muestra información descriptiva de los 500 o 1000 primeros nodos del árbold BnB generado).

In [ ]:
include("lab2TP-BnB-P1.jl");   # Código base para ejercicio 1 del trabajo presencial.
dirInst = "dirTP1";            # Directorio con el archivo de instancias para el ejercicio 1.

function find_branching_variable(sol::Array{Float64,1}, current_node::BBNode)
  # Función que determina el índice s de la variable en la cual bifurcar.
  # Solo se llama cuando sol, una solución factible, es no integral.  
  #
  n_vars = length(sol)
  for s in 1:n_vars
    v = sol[s]
    if ( v > integer_precision ) && ( v < 1 - integer_precision ) && ! is_variable_branched_on(current_node, s)
      return s   # Retorna menor índice fraccional.
    end
  end
  println(" -> Where shall I branch?")    # Warning message.
end

orgn1 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)

  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(orgn1, [profundidad(status.root), nodos(status.root)]);
    
  # Para evitar mensajes de error por generar salidas muy grandes.
  flush(stdout) 
  if isdefined(Main, :IJulia)
    Main.IJulia.stdio_bytes[] = 0
  end
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(orgn1,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn1[i];
  println( " orgn = ", prof1, " / ", nodos1 ) 
end

A continuación, modifique la función `find_branching_varible()` de la última celda para que implemente la estrategia de bifurcación en la variable más fraccional, es decir, asumiendo que las variables del modelo son $x_1,...,x_n$, la citada función deberá retornar el índice $i$ tal que $\min\{x_i,1-x_i\}$ es máximo (en caso de empates retorna el índice menor). Una vez realizada la modificación, ejecute la celda.

In [ ]:
include("lab2TP-BnB-P1.jl");   # Código base para ejercicio 1 del trabajo presencial.
dirInst = "dirTP1";            # Directorio con el archivo de instancias para el ejercicio 1.

function find_branching_variable(sol::Array{Float64,1}, current_node::BBNode)
  # Función que determina el índice s de la variable en la cual bifurcar.
  # Solo se llama cuando sol, una solución factible, es no integral.  
  #
  n_vars = length(sol)
  
  return s    # Retorna índice de la variable más fraccional.
end

mdif1 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)

  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(mdif1, [profundidad(status.root), nodos(status.root)]);
    
  # Para evitar mensajes de error por generar salidas muy grandes.
  flush(stdout) 
  if isdefined(Main, :IJulia)
    Main.IJulia.stdio_bytes[] = 0
  end
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(mdif1,1)
  print( "        ", i, ": ")
  prof2, nodos2 = mdif1[i];
  println( " mdif = ", prof2, " / ", nodos2 ) 
end

Para comparar/resumir el efecto de sus modificaciones a la función `find_branching_variable()`, ejecute la siguiente celda:

In [ ]:
println( "RESUMEN COMPARATIVO:")
println( "Instancia: profunidad / nodos  |  profundidad / nodos" );

for i in 1:size(orgn1,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn1[i];
  prof2, nodos2 = mdif1[i];
  println( " orgn = ", prof1, " / ", nodos1, "   |   mdif = ", prof2, " / ", nodos2 ) 
end

## Ejercico 2: Eliminación de simetrías

Un problema frecuente que se presenta al usar la técnica de BnB es que el problema a resolver presenta muchas simetrías, lo que genera muchísimas repeticiones (módulo permutación de variables) del mismo nodo en el árbol de BnB. Esto no solo ocupa espacio de memoria, sino que fuerza a resolver muchas veces el mismo problema (módulo permutación de variables).

Para esta parte se proveen instancias de $(PCK)$ en que la matriz $A$ y el vector $c$ tienen columnas/coordenadas repetidas (por simplicidad, las columnas/coordenadas repetidas son contiguas). Suponiendo que $J_1,...,J_k$ es una partición de $[n]$, que $c_j$ no depende de $j\in J_i$ y que $A_{\star,j}$ tampoco depende de $j\in J_i$, cualquiera sea $i\in [k]$, es evidente que cualquier $x$ solución factible de $(PCK)$ da lugar (permutando los índices en $J_i$ para $i\in [k]$) a $|J_1|!\cdot...|J_k|!$ soluciones factibles equivalentes.

Ejecute a continuación la siguiente celda y observe el gran número de nodos del árbol BnB que se genera en cada instancia.

In [ ]:
include("lab2TP-BnB-P2.jl");   # Código base para ejercicio 2 del trabajo presencial.
dirInst = "dirTP2";            # Directorio con el archivo de instancias para el ejercicio 2.

function get_lp()
  # Función que crea el modelo a resolver.
  #
  mdl = solver();

  n = size(W,2)-1;  m = size(W,1)-1;
  A = W[1:m,1:n];   b = W[1:m,n+1];  c = W[m+1,1:n];

  # Crear modelo
  @variable(mdl, 1 >= x[1:n] >= 0);
  @constraint(mdl, [i in 1:m], sum( x[j]*A[i,j] for j in 1:n ) <= b[i]);
  @objective(mdl, Max, sum(x .* c) );
  
  return mdl, all_variables(mdl)
end

orgn2 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)

  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(orgn2, [profundidad(status.root), nodos(status.root)]);
    
  # Para evitar mensajes de error por generar salidas muy grandes.
  flush(stdout) 
  if isdefined(Main, :IJulia)
    Main.IJulia.stdio_bytes[] = 0
  end
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(orgn2,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn2[i];
  println( " orgn = ", prof1, " / ", nodos1 ) 
end

2a) Una forma de abordar el problema de las simetrías de una formulación es encontrar otra formulación del problema pero con menos simetrías. Una segunda forma, es agregar restricciones a la formulación para “romper” la simetría. En la siguiente celda, proponga (justifique) qué restricciones agregarle a la formulación $(PCK)$ para reducir lo más posible la cantidad de soluciones factibles equivalentes. 

<div class="alert alert-block alert-warning">
Respuesta Ejercicio 2a:
 
    
    
</div>

2b) Implemente su propuesta modificando el código de la función `get_lp()` en la siguiente celda. Posteriormente, ejecute la celda.

In [ ]:
include("lab2TP-BnB-P2.jl");   # Código base para ejercicio 2 del trabajo presencial.
dirInst = "dirTP2";            # Directorio con el archivo de instancias para el ejercicio

function get_lp()
  # Función que crea el modelo a resolver.
  # 
  mdl = solver();

  n = size(W,2)-1;  m = size(W,1)-1;
  A = W[1:m,1:n];   b = W[1:m,n+1];  c = W[m+1,1:n];

  # Crear modelo
  @variable(mdl, 1 >= x[1:n] >= 0);
  @constraint(mdl, [i in 1:m], sum( x[j]*A[i,j] for j in 1:n ) <= b[i]);
  @objective(mdl, Max, sum(x .* c) );

  # Begin: Restricciones para romper simetrías
  #  
  
  # End: Restricciones para romper simetrías
  
  return mdl, all_variables(mdl)
end

mdif2 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)
  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(mdif2, [profundidad(status.root), nodos(status.root)]);
    
  # Para evitar mensajes de error por generar salidas muy grandes.
  flush(stdout) 
  if isdefined(Main, :IJulia)
    Main.IJulia.stdio_bytes[] = 0
  end    
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(mdif2,1)
  print( "        ", i, ": ")
  prof1, nodos1 = mdif2[i];
  println( " mdif = ", prof1, " / ", nodos1 ) 
end

Para comparar el efecto de sus modificaciones a la función `get_lp()`, ejecute la siguiente celda:

In [ ]:
println( "RESUMEN COMPARATIVO:")
println( "Instancia: profunidad / nodos  |  profundidad / nodos" );
for i in 1:size(orgn2,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn2[i];
  prof2, nodos2 = mdif2[i];
  println( " orgn = ", prof1, " / ", nodos1, "   |   mdif = ", prof2, " / ", nodos2 ) 
end

## Ejercicio 3: *Warm/cold-start* en BnB

Cuando uno inicializa el proceso de BnB con una solución factible  se habla de *warm-start* (en contraposición a *cold-start* cuando no se tiene una solución factible inicial). Mientras mejor la solución factible (su valor este más próximo al óptimo) mejor es la cota inicial disponible y más efectivo el proceso de poda del árbol de BnB.

Entre las estrategias que se usan para construir buenas soluciones factibles iniciales para un *warm-start* están: estrategias glotonas, aleatoriedad, heurísticas, etc.

No es del todo obvio como construir una solución factible inicial para ciertos problemas. En particular en el caso de $(PCK)$. En general, para desarrollar heurísticas para construir soluciones factibles iniciales ayuda el tener información más precisa sobre el tipo de instancias de interés (no obstante, a menudo esta información
no está disponible).

El conjunto de instancias que se proveen para esta parte corresponden a casos de $(PCK)$ donde $m=n$. Cada uno de los coeficientes $A_{i,j}$ fue elegido uniforme e independientemente en $\{1,2,...,20\}$ mientras que las coordenadas de $c$ fueron elegidas uniforme e independientemente en $\{10,11,...,60\}$. La coordenada $i$-ésima de $b$ se calculó como
$$
b_i = \sum_{j\in [n]} A_{i,j}X_{i,j}-1,
$$
donde las $X_{i,j}$'s son variables aleatorias independientes equidistribuidas a valores en $0$-$1$ (i.e., tipo Bernoulli) con esperanza $1/10$.

Ejecute a continuación la siguiente celda, que implementa un *cold-start* de BnB para $(PCK)$ en las instancias proveídas. Observe el número de nodos del árbol BnB que se genera en cada caso.

In [ ]:
include("lab2TP-BnB-P3.jl");   # Código base para ejercicio 3 del trabajo presencial.
dirInst = "dirTP3";            # Directorio con el archivo de instancias para el ejercicio

function get_root_incumbent()
  # Función que permite retornar una solución factible inicial (incumbente)
  # para un warm-start (en caso contrario retorna [], +/-Inf).
  # 
  return [], -Inf
end

orgn3 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)
  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(orgn3, [profundidad(status.root), nodos(status.root)]);

  # Para evitar mensajes de error por generar salidas muy grandes.
  flush(stdout) 
  if isdefined(Main, :IJulia)
    Main.IJulia.stdio_bytes[] = 0
  end  
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(orgn3,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn3[i];
  println( " orgn = ", prof1, " / ", nodos1 ) 
end

3a) En la siguiente celda, proponga (justifique) para construir una potencialmente “buena” solución factible inicial para un *warm-start* en las instancias de $(PCK)$ como las proveídas para este ejercicio.

<div class="alert alert-block alert-warning">
Respuesta Ejercicio 3a:
    
    
    
</div>

3b) A continuación, modifique la función `get_root_incumbent()` para que implemente su propuesta para el *warm-start*. Su función debe retornar un arreglo representando $x\in\{0,1\}^n$ solución factible inicial (supuestamente “buena”) de la instancia de $(PCK)$ y el valor $\sum_{i\in [n]}c_ix_i$ de dicha solución. 

In [ ]:
include("lab2TP-BnB-P3.jl");   # Código base para ejercicio 3 del trabajo presencial.
dirInst = "dirTP3";            # Directorio con el archivo de instancias para el ejercicio

function get_root_incumbent()
  # Función que permite retornar una solución factible inicial (incumbente)
  # para un warm-start  (en caso contrario retorna [], +/-Inf).
  # 
  n = size(W,2)-1;   m = size(W,1)-1;
  A = W[1:m,1:n];    b = W[1:m,n+1];    c = W[m+1,1:n];

  # Begin: Determina solución factible inicial
    
  # End: Determinar solución factible inicial
    
  # Return initial solution and its value
  return x, sum( c.*x )
end

mdif3 = [];
for fn in readdir(dirInst)
  W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( "out ", repeat('=', 60) );
  println( "out Instancia ", fn );
  println( "out ", repeat('=', 60) );
  status = bnb()
  print(status)
  println("out tree up/down bounds ", status.upper_bound, "/", status.lower_bound )

  # Guarda estadísticas para análisis posterior.
  push!(mdif3, [profundidad(status.root), nodos(status.root)]);
end

# Imprime características de los árboles BnB generados para cada instancia.
println( "" ); println( "RESUMEN:")
println( "Instancia: profunidad / nodos" );
for i in 1:size(mdif3,1)
  print( "        ", i, ": ")
  prof1, nodos1 = mdif3[i];
  println( " mdif = ", prof1, " / ", nodos1 ) 
end

Compare los resultados obtenidos en ambas ejecuciones del programa con y sin modificaciones. Para al menos una de las instancias que se proveen, su programa debe generar árboles de BnB con una menor cantidad de nodos que los árboles correspondientes del *cold-start*.

In [ ]:
println( "RESUMEN COMPARATIVO:")
println( "Instancia: profunidad / nodos  |  profundidad / nodos" );
for i in 1:size(orgn3,1)
  print( "        ", i, ": ")
  prof1, nodos1 = orgn3[i];
  prof2, nodos2 = mdif3[i];
  println( " orgn = ", prof1, " / ", nodos1, "   |   mdif = ", prof2, " / ", nodos2 ) 
end